In [45]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
## you will need to use the properties_2017 and predictions_2017 tables.

## square feet of the house ("living square feet"), number of bedrooms, number of bathrooms, the assess value of the house by the tax appraisal district ('taxvaluedollarcnt'...this will be your target variable in the project), and 2-3 other variables

## may, june, july or august (date is in predictions_2017 table)

## single unit property values

In [4]:
sql = '''
select parcelid, calculatedfinishedsquarefeet as square_feet, bedroomcnt as bedrooms, bathroomcnt as bathrooms, taxamount as taxes, yearbuilt, regionidcounty as county, lotsizesquarefeet as lot_size, taxvaluedollarcnt as tax_value
from properties_2017
join predictions_2017 using(parcelid)
where transactiondate between "2017-05-01" and "2017-08-31"
AND propertylandusetypeid > 250
AND propertylandusetypeid < 280 
AND propertylandusetypeid != 270 
AND propertylandusetypeid != 271
OR  unitcnt = 1;
'''

In [5]:
df = pd.read_sql(sql, get_connection('zillow'))

In [6]:
df.head()

,parcelid,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size,tax_value
0,12177905,2376.0,4.0,3.0,1777.51,1970.0,3101.0,13038.0,145143.0
1,10887214,1312.0,3.0,3.0,1533.89,1964.0,3101.0,278581.0,119407.0
2,12095076,2962.0,4.0,3.0,9516.26,1950.0,3101.0,63000.0,773303.0
3,12069064,738.0,2.0,1.0,2366.08,1922.0,3101.0,4214.0,218552.0
4,12790562,3039.0,4.0,3.0,3104.19,1970.0,3101.0,20028.0,220583.0


In [7]:
df.shape

(62159, 9)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62159 entries, 0 to 62158
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   parcelid     62159 non-null  int64  
 1   square_feet  62040 non-null  float64
 2   bedrooms     62159 non-null  float64
 3   bathrooms    62159 non-null  float64
 4   taxes        62155 non-null  float64
 5   yearbuilt    62015 non-null  float64
 6   county       62159 non-null  float64
 7   lot_size     57158 non-null  float64
 8   tax_value    62158 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 4.3 MB


In [9]:
print(df.isnull().sum())

parcelid          0
square_feet     119
bedrooms          0
bathrooms         0
taxes             4
yearbuilt       144
county            0
lot_size       5001
tax_value         1
dtype: int64


In [10]:
print(df.columns[df.isnull().any()])

Index(['square_feet', 'taxes', 'yearbuilt', 'lot_size', 'tax_value'], dtype='object')


In [11]:
df.to_csv("zillow.csv") 

In [12]:
df = df.dropna()

In [13]:
df.shape

(57057, 9)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57057 entries, 0 to 62158
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   parcelid     57057 non-null  int64  
 1   square_feet  57057 non-null  float64
 2   bedrooms     57057 non-null  float64
 3   bathrooms    57057 non-null  float64
 4   taxes        57057 non-null  float64
 5   yearbuilt    57057 non-null  float64
 6   county       57057 non-null  float64
 7   lot_size     57057 non-null  float64
 8   tax_value    57057 non-null  float64
dtypes: float64(8), int64(1)
memory usage: 4.4 MB


In [15]:
print(df.isnull().sum())

parcelid       0
square_feet    0
bedrooms       0
bathrooms      0
taxes          0
yearbuilt      0
county         0
lot_size       0
tax_value      0
dtype: int64


In [16]:
df.set_index("parcelid")

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size,tax_value
parcelid,,,,,,,,
12177905,2376.0,4.0,3.0,1777.51,1970.0,3101.0,13038.0,145143.0
10887214,1312.0,3.0,3.0,1533.89,1964.0,3101.0,278581.0,119407.0
12095076,2962.0,4.0,3.0,9516.26,1950.0,3101.0,63000.0,773303.0
12069064,738.0,2.0,1.0,2366.08,1922.0,3101.0,4214.0,218552.0
12790562,3039.0,4.0,3.0,3104.19,1970.0,3101.0,20028.0,220583.0
...,...,...,...,...,...,...,...,...
10833991,1741.0,3.0,3.0,4685.34,1980.0,3101.0,59487.0,379000.0
11000655,1286.0,2.0,2.0,4478.43,1940.0,3101.0,47405.0,354621.0
12773139,1032.0,3.0,1.0,876.43,1954.0,3101.0,5074.0,49546.0


In [17]:
def wrangle_zillow():
    data = pd.read_csv("zillow.csv")
    
    data = data.drop(columns = 'Unnamed: 0')
    
    data = data.set_index("parcelid")
    
    data = data.dropna()
    # remove all NaN values
    
    return data

In [18]:
def split(df, stratify_by=None):
    """
    Crude train, validate, test split
    To stratify, send in a column name
    """
    
    if stratify_by == None:
        train, test = train_test_split(df, test_size=.2, random_state=319)
        train, validate = train_test_split(train, test_size=.3, random_state=319)
    else:
        train, test = train_test_split(df, test_size=.2, random_state=319, stratify=df[stratify_by])
        train, validate = train_test_split(train, test_size=.3, random_state=319, stratify=train[stratify_by])
    
    return train, validate, test

In [19]:
df2 = wrangle_zillow()

In [20]:
df2.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size,tax_value
parcelid,,,,,,,,
12177905,2376.0,4.0,3.0,1777.51,1970.0,3101.0,13038.0,145143.0
10887214,1312.0,3.0,3.0,1533.89,1964.0,3101.0,278581.0,119407.0
12095076,2962.0,4.0,3.0,9516.26,1950.0,3101.0,63000.0,773303.0
12069064,738.0,2.0,1.0,2366.08,1922.0,3101.0,4214.0,218552.0
12790562,3039.0,4.0,3.0,3104.19,1970.0,3101.0,20028.0,220583.0


In [21]:
df2.shape

(57057, 8)

In [22]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57057 entries, 12177905 to 13083743
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   square_feet  57057 non-null  float64
 1   bedrooms     57057 non-null  float64
 2   bathrooms    57057 non-null  float64
 3   taxes        57057 non-null  float64
 4   yearbuilt    57057 non-null  float64
 5   county       57057 non-null  float64
 6   lot_size     57057 non-null  float64
 7   tax_value    57057 non-null  float64
dtypes: float64(8)
memory usage: 3.9 MB


In [23]:
df2.describe()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size,tax_value
count,57057.000000,57057.000000,57057.000000,57057.000000,57057.000000,57057.000000,5.705700e+04,5.705700e+04
mean,1765.189267,3.044517,2.270834,6000.181325,1966.119565,2796.600452,3.445027e+04,4.864494e+05
std,946.854611,1.011406,0.975278,7738.204208,23.077203,647.646948,1.347859e+05,6.622019e+05
min,152.000000,0.000000,0.000000,19.920000,1878.000000,1286.000000,2.360000e+02,1.000000e+03
25%,1175.000000,2.000000,2.000000,2627.890000,1951.000000,3101.000000,5.876000e+03,1.934810e+05
50%,1527.000000,3.000000,2.000000,4374.100000,1965.000000,3101.000000,7.442000e+03,3.458950e+05
75%,2084.000000,4.000000,3.000000,6863.030000,1984.000000,3101.000000,1.406200e+04,5.632820e+05
max,35640.000000,11.000000,13.000000,290998.060000,2016.000000,3101.000000,6.971010e+06,2.538125e+07


In [24]:
train, validate, test = split(df2)

In [25]:
train.shape

(31951, 8)

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31951 entries, 12794645 to 10902239
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   square_feet  31951 non-null  float64
 1   bedrooms     31951 non-null  float64
 2   bathrooms    31951 non-null  float64
 3   taxes        31951 non-null  float64
 4   yearbuilt    31951 non-null  float64
 5   county       31951 non-null  float64
 6   lot_size     31951 non-null  float64
 7   tax_value    31951 non-null  float64
dtypes: float64(8)
memory usage: 2.2 MB


In [27]:
validate.shape

(13694, 8)

In [28]:
test.shape

(11412, 8)

In [29]:
def scale_data(train, validate, test):
    
    '''
    This function will scale numeric data using Min Max transform after 
    it has already been split into train, validate, and test.
    '''
    
    # Make the thing
    scaler = sklearn.preprocessing.MinMaxScaler()
    
    # We fit on the training data
    # we only .fit on the training data
    scaler.fit(train)
    
    train_scaled = scaler.transform(train)
    validate_scaled = scaler.transform(validate)
    test_scaled = scaler.transform(test)
    
    # turn the numpy arrays into dataframes
    train_scaled = pd.DataFrame(train_scaled, columns=train.columns)
    validate_scaled = pd.DataFrame(validate_scaled, columns=train.columns)
    test_scaled = pd.DataFrame(test_scaled, columns=train.columns)
    
    return train_scaled, validate_scaled, test_scaled

In [30]:
X_train = train.drop(columns=['tax_value'])
y_train = train.tax_value

X_validate = validate.drop(columns=['tax_value'])
y_validate = validate.tax_value

X_test = test.drop(columns=['tax_value'])
y_test = test.tax_value

In [31]:
X_train.shape

(31951, 7)

In [32]:
y_train.shape

(31951,)

In [33]:
def seperate_y(train, validate, test):
    X_train = train.drop(columns=['tax_value'])
    y_train = train.tax_value

    X_validate = validate.drop(columns=['tax_value'])
    y_validate = validate.tax_value

    X_test = test.drop(columns=['tax_value'])
    y_test = test.tax_value
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

In [34]:
X_train, y_train, X_validate, y_validate, X_test, y_test = seperate_y(train, validate, test)

In [35]:
X_train.shape

(31951, 7)

In [36]:
y_train.shape

(31951,)

In [37]:
X_validate.shape

(13694, 7)

In [38]:
y_validate.shape

(13694,)

In [39]:
X_test.shape

(11412, 7)

In [40]:
y_test.shape

(11412,)

In [41]:
X_train.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
parcelid,,,,,,,
12794645,1467.0,3.0,2.0,1206.35,1958.0,3101.0,9743.0
11150935,2086.0,3.0,3.0,7874.20,2005.0,3101.0,19215.0
12819463,1472.0,4.0,3.0,3693.69,1954.0,3101.0,5517.0
11621154,1800.0,2.0,2.0,8455.09,1974.0,3101.0,25059.0
12772236,968.0,3.0,1.0,4954.28,1954.0,3101.0,5118.0


In [42]:
X_validate.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
parcelid,,,,,,,
12979856,984.0,2.0,1.0,2159.23,1953.0,3101.0,5937.0
14098411,1673.0,4.0,2.0,6571.12,1968.0,1286.0,5005.0
12745593,1440.0,4.0,2.0,1086.60,1963.0,3101.0,5029.0
14362589,1919.0,4.0,2.5,7925.90,1993.0,1286.0,4600.0
12070788,958.0,1.0,1.0,3795.89,1975.0,3101.0,40709.0


In [43]:
X_test.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
parcelid,,,,,,,
12197745,1445.0,2.0,1.0,593.73,1931.0,3101.0,5743.0
12538651,655.0,1.0,1.0,1066.94,1957.0,3101.0,15001.0
13973106,1493.0,3.0,1.0,3084.40,1952.0,1286.0,6018.0
11887550,3070.0,3.0,3.0,14967.95,1950.0,3101.0,21944.0
11420117,1194.0,3.0,2.0,3283.20,1954.0,3101.0,5824.0


In [46]:
train_scaled, validate_scaled, test_scaled = scale_data(X_train, X_validate, X_test)

In [47]:
train_scaled.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
0,0.056572,0.3,0.153846,0.003970,0.579710,1.0,0.001335
1,0.085112,0.3,0.230769,0.026888,0.920290,1.0,0.002694
2,0.056803,0.4,0.230769,0.012519,0.550725,1.0,0.000729
3,0.071926,0.2,0.153846,0.028884,0.695652,1.0,0.003533
4,0.033565,0.3,0.076923,0.016852,0.550725,1.0,0.000672


In [48]:
validate_scaled.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
0,0.034303,0.2,0.076923,0.007245,0.543478,1.0,0.000789
1,0.066070,0.4,0.153846,0.022409,0.652174,0.0,0.000656
2,0.055328,0.4,0.153846,0.003559,0.615942,1.0,0.000659
3,0.077413,0.4,0.192308,0.027066,0.833333,0.0,0.000598
4,0.033104,0.1,0.076923,0.012870,0.702899,1.0,0.005778


In [49]:
test_scaled.head()

,square_feet,bedrooms,bathrooms,taxes,yearbuilt,county,lot_size
0,0.055558,0.2,0.076923,0.001864,0.384058,1.0,0.000761
1,0.019134,0.1,0.076923,0.003491,0.572464,1.0,0.002090
2,0.057771,0.3,0.076923,0.010425,0.536232,0.0,0.000801
3,0.130481,0.3,0.230769,0.051269,0.521739,1.0,0.003086
4,0.043985,0.3,0.153846,0.011108,0.550725,1.0,0.000773
